In [1]:

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import os
import torch.nn.functional as F
import torch.optim as optim
from ResNet import ResNet50
from split_asl_alphabet.dataset import ASLDataset

In [2]:
GPUS = [0]
EPOCHS = 10
NO_CLASSES = 29
TRAIN_DIR = 'split_asl_alphabet/train'
VAL_DIR = 'split_asl_alphabet/test'
IMG_DIM = 226
BATCH_SIZE = 128
L_RATE = 0.01
W_DECAY = 0.0005
MOMENTUM = 0.9
CHECKPOINT_DIR = 'checkpoints'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
data_transform = transforms.Compose([
    transforms.Resize(IMG_DIM),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5,0.5], std=[0.5,0.5,0.5])
])

In [4]:
train_data_simple = ASLDataset(data_path=TRAIN_DIR,range_index = [1,2500], transform=data_transform)
test_data_simple = ASLDataset(data_path=TRAIN_DIR,range_index = [2500,3000], transform=data_transform)

# Setup batch size and number of workers 
NUM_WORKERS = os.cpu_count()
print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")

# Create DataLoader's
train_dataloader_simple = DataLoader(train_data_simple, 
                                     batch_size=BATCH_SIZE, 
                                     shuffle=True, 
                                     num_workers=NUM_WORKERS)

test_dataloader_simple = DataLoader(test_data_simple, 
                                    batch_size=BATCH_SIZE, 
                                    shuffle=False, 
                                    num_workers=NUM_WORKERS)
print(f'train dataset size :{len(train_data_simple)} images ')
print(f'test dataset size :{len(test_data_simple)} images ')

Creating DataLoader's with batch size 128 and 20 workers.
train dataset size :48488 images 
test dataset size :9483 images 


In [5]:
net = ResNet50(29).to('cpu')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [19]:
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for i, inp in enumerate(train_dataloader_simple):
        inputs= inp
        inputs, labels = inputs.to('cpu'), labels.to('cpu')
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')

AttributeError: 'list' object has no attribute 'to'

In [7]:
total_steps = 1
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for step, inpt in enumerate(train_dataloader_simple):
        # Assuming inpt is a tuple with input data and labels
        X, y = inpt
        X, y = X.to(device), y.to(device)
        
        # refreshing gradients
        optimizer.zero_grad()
        
        # forward pass
        pred = net(X)
        
        # calculate the loss
        loss = criterion(pred, y)
        losses.append(loss.item())
        
        # backward pass
        loss.backward()
        
        # update weights
        optimizer.step()
        
        running_loss += loss.item()
        
        if total_steps % 10 == 0:
            print(f'Loss [{epoch+1}, {step+1}](epoch, minibatch): ', running_loss / 10)
            running_loss = 0.0
        
        total_steps += 1
    
    avg_loss = sum(losses) / len(losses)
    scheduler.step(avg_loss)

print('Training Done')


ValueError: too many values to unpack (expected 2)

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_dataloader_simple:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')